In [1]:
import cv2
import numpy as np
import sys
import dlib
import scipy.spatial as sp

In [2]:
#Read points from  text file
def readPoints(path):
    # Create an array of points
    points = []
    # Read points
    with open(path) as file:
        for line in file:
            x,y = line.split()
            points.append((int(x),int(y)))

    return points

In [3]:
# Apply affine tranform calculated using srcTri and sdtTri to src and output an image of size
def applyAffineTransform(src,srcTri,dstTri,size):

    #Given a pair of triangles,find the affine transform.

    warpMat = cv2.getAffineTransform(np.float32(srcTri),np.float32(dstTri))

    #Apply the Affine Transform just foundto the src image
    dst = cv2.warpAffine(src,warpMat,(size[0],size[1]),None,flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REFLECT_101)

    return dst

In [4]:
# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1,img2,img,t1,t2,t,alpha):

    # Find bounding rectangle for each triangle
    # 用一個最小的矩形，把五官的points都包起來
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []

    for i in range(0,3):
        tRect.append(((t[i][0] - r[0]),(t[i][1]-r[1])))
        t1Rect.append(((t1[i][0]-r1[0]),(t1[i][1]-r1[1])))
        t2Rect.append(((t2[i][0] -r2[0]),(t2[i][1]-r2[1])))

    # Get mask by filling triangles
    mask = np.zeros((r[3],r[2],3),dtype = np.float32)
    cv2.fillConvexPoly(mask,np.int32(tRect),(1.0,1.0,1.0),16,0)

    # Apply warpImage to small rectangular patched
    img1Rect = img1[r1[1]:r1[1]+r1[3],r1[0]:r1[0]+r1[2]]
    img2Rect = img2[r2[1]:r2[1]+r2[3],r2[0]:r2[0]+r2[2]]

    size = (r[2],r[3])
    warpImage1 = applyAffineTransform(img1Rect,t1Rect,tRect,size)
    warpImage2 = applyAffineTransform(img2Rect,t2Rect,tRect,size)

    # Alpha blend rectangular patches
    imgRect = (1.0-alpha) *warpImage1 +alpha*warpImage2

    # Copy triangular region of rectangular patch to tje output image
    print(r[1],r[3],r[0],r[2])
    print(imgRect.shape)
    img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]*(1-mask) +imgRect*mask


In [ ]:
if __name__ =='__main__':
    filename1 = "./models/girl.PNG"
    filename2 = "./models/boy.PNG"

    points_txt1 = "./location/img1_68.txt"
    points_txt2  ="./location/img2_68.txt"

    alpha = 0.5

    # Read images
    img1 = cv2.imread(filename1)
    img2 = cv2.imread(filename2)

    # Convertat to float data type
    img1 = np.float32(img1)
    img2 = np.float32(img2)

    # Read array of corresponding points
    points1 = readPoints(points_txt1)
    points2 = readPoints(points_txt2)
    points = []


    # Compute weighted average point coordinate

    for i in range(0,len(points1)):
        x = (1-alpha) *points1[i][0] +alpha *points2[i][0]
        y = (1-alpha)*points1[i][1] + alpha*points2[i][1]
        points.append((x,y))


    imgMorph = np.zeros(img1.shape,dtype = img1.dtype)
    # 計算三角測量索引
    dela = sp.Delaunay
    triang = dela(points)
    for tri in triang.vertices:
        x = tri[0]
        y = tri[1]
        z = tri[2]
        t1 = [points1[x],points1[y],points1[z]]
        t2 = [points2[x],points2[y],points2[z]]
        t = [points[x],points[y],points[z]]
        # Morph one triangle at a time
        morphTriangle(img1,img2,imgMorph,t1,t2,t,alpha)

    # Display Results

    out_img = np.hstack((img1,imgMorph,img2))
    cv2.imwrite("./output/morph_final.jpg",imgMorph)
    cv2.imwrite("./output/morph_all.jpg",out_img)
#     cv2.imshow("Morphed Face",out_img)
#     cv2.imshow("Morphed Face",np.uint8(imgMorph))
    cv2.waitKey(0)


113 16 152 22
(16, 22, 3)
116 13 148 26
(13, 26, 3)
108 15 67 16
(15, 16, 3)
163 13 74 25
(13, 25, 3)
122 20 94 23
(20, 23, 3)
120 22 116 25
(22, 25, 3)
107 14 115 19
(14, 19, 3)
103 14 152 22
(14, 22, 3)
99 18 152 12
(18, 12, 3)
102 6 97 38
(6, 38, 3)
99 6 89 56
(6, 56, 3)
102 4 89 46
(4, 46, 3)
121 16 67 16
(16, 16, 3)
180 18 86 20
(18, 20, 3)
173 15 86 20
(15, 20, 3)
173 15 79 20
(15, 20, 3)
157 17 74 35
(17, 35, 3)
150 14 70 39
(14, 39, 3)
141 17 108 9
(17, 9, 3)
124 34 70 39
(34, 39, 3)
124 34 94 23
(34, 23, 3)
141 16 116 11
(16, 11, 3)
121 36 116 25
(36, 25, 3)
157 17 98 11
(17, 11, 3)
182 26 119 14
(26, 14, 3)
182 23 123 22
(23, 22, 3)
109 14 101 15
(14, 15, 3)
105 5 97 29
(5, 29, 3)
107 13 106 20
(13, 20, 3)
120 22 116 18
(22, 18, 3)
119 12 115 19
(12, 19, 3)
122 20 101 16
(20, 16, 3)
119 12 101 16
(12, 16, 3)
99 6 80 65
(6, 65, 3)
103 15 80 10
(15, 10, 3)
108 14 72 16
(14, 16, 3)
103 15 72 16
(15, 16, 3)
99 16 134 11
(16, 11, 3)
114 8 133 8
(8, 8, 3)
107 14 125 15
(14, 15, 3)
